In [ ]:
import os, cv2, numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import visualkeras

import tensorflow as tf
from tensorflow.keras.layers import MaxPooling2D, Conv2D, Input, Flatten, Dense, Dropout
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import utils

from imgaug import augmenters as iaa
import imgaug as ia
ia.seed(42)

In [ ]:
def save_result(val_out, val_block_size, img_size, max_num, image_path):
    def preprocess(img):
        img = (img*255.0).astype(np.uint8)
        img = np.squeeze(img, axis=0)
        return img

    preprocesed = preprocess(val_out)
    # print(preprocesed.shape)
    final_image = np.array([])
    single_row = np.array([])
    if max_num > val_out.shape[-1]:
        num = val_out.shape[-1]
    else:
        num = max_num
    if val_block_size > num:
        print("val_block_size必須小於num")
        return
    for b in range(num):
        # concat image into a row
        if single_row.size == 0:
            single_row = preprocesed[ :, :, b]
            single_row = cv2.resize(single_row, (img_size, img_size))
        else:
            single_row_now = preprocesed[:, :, b]
            single_row_now = cv2.resize(single_row_now, (img_size, img_size))
            single_row = np.concatenate((single_row, single_row_now), axis=1)

        # concat image row to final_image
        if (b+1) % val_block_size == 0:
            if final_image.size == 0:
                final_image = single_row
            else:
                final_image = np.concatenate((final_image, single_row), axis=0)

            # reset single row
            single_row = np.array([])
    final_image = cv2.cvtColor(final_image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(image_path, final_image)

In [ ]:
def lr_scheduler(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-4
    if epoch > 80:
        lr *= 0.5e-3
    elif epoch > 40:
        lr *= 1e-3
    elif epoch > 20:
        lr *= 1e-2
    elif epoch > 10:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
def CNN():
    inputs = Input(shape=(32, 32, 3))
    conv1 = Conv2D(filters=32, strides=(1, 1), kernel_size=3, input_shape=(32, 32, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(filters=32, strides=(1, 1), kernel_size=3, activation='relu', padding='same')(conv1)
    MaxPool2D1 = MaxPooling2D(pool_size=2)(conv1)

    conv2 = Conv2D(filters=64, strides=(1, 1), kernel_size=3, activation='relu', padding='same')(MaxPool2D1)
    conv2 = Conv2D(filters=64, strides=(1, 1), kernel_size=3, activation='relu', padding='same')(conv2)
    MaxPool2D2 = MaxPooling2D(pool_size=2)(conv2)
    
    conv3 = Conv2D(filters=128, strides=(1, 1), kernel_size=3, activation='relu', padding='same')(MaxPool2D2)
    conv3 = Conv2D(filters=128, strides=(1, 1), kernel_size=3, activation='relu', padding='same')(conv3)
    MaxPool2D3 = MaxPooling2D(pool_size=2)(conv3)

    
    conv4 = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(MaxPool2D3)
    conv4 = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(conv4)
    MaxPool2D4 = MaxPooling2D(pool_size=2)(conv4)
    
    '''conv5 = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(MaxPool2D4)
    conv5 = Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(conv5)
    MaxPool2D5 = MaxPooling2D(pool_size=2)(conv5)'''
    

    faltten1 = Flatten()(MaxPool2D4)

    dense1 = Dense(512, activation='relu')(faltten1)
    dropout4 = Dropout(rate=0.35)(dense1)
    out = Dense(10, activation='softmax')(dropout4)     # softmax is important !!

    model = Model(inputs, out, name = "cnn")
    model.compile(loss='categorical_crossentropy', 
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler(0)), # 0.0001
                  metrics=['accuracy'])

    return model

In [ ]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [ ]:
X_train.shape

In [ ]:
'''
rand_aug = iaa.SomeOf((0, 2), [
    iaa.Crop(px=(1, 16), keep_size=True),
    iaa.Fliplr(0.5),
    iaa.GaussianBlur(sigma=(0, 3.0))
])
'''

In [ ]:
rand_aug = iaa.RandAugment(n=7, m=3)
images_aug = rand_aug(images=X_train)

In [ ]:
print('X_train\'s shape  =   ', X_train.shape)
print('images_aug\'s shape = ', X_train.shape)

In [ ]:
# rand_aug.show_grid([images_aug[0], images_aug[1]], cols=8, rows=4)

In [ ]:
'''
xx = 5
plt.subplot(1, 2, 1)
plt.title("Original")
plt.imshow(np.array(X_train[xx]))

plt.subplot(1, 2, 2)
plt.title("SimpleAug")
plt.imshow(np.array(images_aug[xx]))
'''

In [ ]:
X_train_aug = np.concatenate([X_train, np.array(images_aug)])
Y_train_aug = np.concatenate([Y_train, Y_train])

x_train_aug = X_train_aug.astype('float32')/255.0
x_test = X_test.astype('float32')/255.0

# One-hot encoding
y_train_aug = utils.to_categorical(Y_train_aug)
y_test = utils.to_categorical(Y_test)

print(x_train_aug.shape, y_train_aug.shape)

In [ ]:
checkpoint = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

for kfold, (train, valid) in enumerate(KFold(n_splits=6, shuffle=True).split(x_train_aug, y_train_aug)):
    # clear the session 
    tf.keras.backend.clear_session()

    # calling the model and compile it 
    model = CNN()

    print('------------------------------------------')
    print('            Fold ' + str(kfold+1) + ' processing')
    print('------------------------------------------')
    
    # run the model 
    history = model.fit(x_train_aug[train], y_train_aug[train], 
                        batch_size=32, 
                        epochs=100, 
                        validation_data=(x_train_aug[valid], y_train_aug[valid]), 
                        callbacks=[checkpoint, LearningRateScheduler(lr_scheduler)],
                        shuffle=True, 
                        verbose=1)
    # seq_model.save_weights(f'wg_{kfold}.h5')

In [ ]:
def show_train_history(train_history, title, train, validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title(title)
    plt.ylabel('train')
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='center right')
    plt.show()

In [ ]:
show_train_history(history, 'Train History', 'accuracy', 'val_accuracy')
show_train_history(history, 'Loss History', 'loss', 'val_loss')

In [ ]:
visualkeras.layered_view(model, legend=True)

In [ ]:
# result
loss, accuracy = model.evaluate(x_test, y_test)
print('Test:')
print('Loss:', loss)
print('Accuracy:', accuracy)

In [ ]:
# save process img
'''cv2.imwrite('test.jpg', X_train[10])
s = cv2.imread('test.jpg')
img = cv2.resize(s, (32, 32))
img_batch = np.expand_dims(img, axis=0)
   
for layer_name in ['conv2d', 'conv2d_1', 'max_pooling2d', 'conv2d_2', 'conv2d_3', 'max_pooling2d_1', 'conv2d_4', 'conv2d_5', 'max_pooling2d_2']:
    conv1_layer = layer_model(model, layer_name)
    conv_img = conv1_layer.predict(img_batch)
    save_result(conv_img, 12, 32, 64, './' + 'Layer_img_' + layer_name + '.jpg')'''